In [1]:
import sys
sys.path.insert(1, "../")
import utilities as utils

from datetime import datetime
import pandas as pd

from sklearn import tree
import graphviz

import pycaret.anomaly as py_caret_anom
import pycaret.classification as py_caret_class
from collections import Counter

from tqdm.notebook import tqdm

import altair as alt
alt.data_transformers.enable("vegafusion")

ImportError: cannot import name '_print_elapsed_time' from 'sklearn.utils' (X:\__Anaconda3__\envs\OutlierEnv\lib\site-packages\sklearn\utils\__init__.py)

In [ ]:
# tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.27.2 which is incompatible.
# tensorflow 2.10.1 requires keras<2.11,>=2.10.0, but you have keras 3.4.1 which is incompatible.
# tensorflow 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.27.2 which is incompatible.
# tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.10.1 which is incompatible.
# matplotlib 3.7.5 requires numpy<2,>=1.20, but you have numpy 2.0.0 which is incompatible.
# pycaret 3.3.2 requires joblib<1.4,>=1.2.0, but you have joblib 1.4.2 which is incompatible.
# pycaret 3.3.2 requires numpy<1.27,>=1.21, but you have numpy 2.0.0 which is incompatible.
# pycaret 3.3.2 requires pandas<2.2.0, but you have pandas 2.2.2 which is incompatible.
# pycaret 3.3.2 requires scipy<=1.11.4,>=1.6.1, but you have scipy 1.13.1 which is incompatible.
# sktime 0.26.0 requires numpy<1.27,>=1.21, but you have numpy 2.0.0 which is incompatible.
# sktime 0.26.0 requires pandas<2.2.0,>=1.1, but you have pandas 2.2.2 which is incompatible.
# sktime 0.26.0 requires scikit-learn<1.5.0,>=0.24, but you have scikit-learn 1.5.1 which is incompatible.

In [ ]:
# !pip install "pandas<2.2.0"

In [ ]:
start_time = datetime.now()
print(f" Process started @ {start_time}")

In [ ]:
# !pip install graphviz

In [ ]:
global_excluded_cols = ['Field03', 
                        'Field30',
                        'Field91',
                        'Field92',
                        'Field93',
                        'Field94',
                        'Field95',
                        'Field96']

# global_excluded_cols = ["DateKey", 
#                         'TruckOperationCostRate',
#                         'DiggerOperationCostRate',
#                         'TruckOperationCost',
#                         'DiggerOperationCost',
#                         'CycleOperationCost',
#                         'Revenue']
random_state = 423546 # 21435 # 218
null_threshold = 0.01
anomaly_fraction = 0.35 # 0.01 0.05 0.10 0.15 0.20 0.25 0.30 0.35 
anomaly_fractions = [0.01, 0.35] # [0.01, 0.15, 0.35]
max_encoding_ohe = 26 # 11
lime_num_features = 5

color_scheme = "plasma" # plasma turbo yellowgreenblue goldorange

In [ ]:
sample_data_df = pd.read_csv("../output/sample_data_00.csv")
sample_data_df

In [ ]:
# Setting the DataFrame index
possible_unique_key_list = utils.get_unique_col(source_df=sample_data_df)

if len(possible_unique_key_list) == 1:
    alt_sample_data_df = sample_data_df.copy()
    alt_sample_data_df = alt_sample_data_df.set_index(possible_unique_key_list[0])
else:
    print(f"Could not detect index column automatically. Using default.")
    alt_sample_data_df = sample_data_df.copy()
    alt_sample_data_df = alt_sample_data_df.set_index(["Field00"])

In [ ]:
alt_sample_data_df

In [ ]:
alt_sample_data_df = alt_sample_data_df.drop(columns=global_excluded_cols)

In [ ]:
alt_sample_data_df

In [ ]:
# Dropping rows with missing values
perc_null_dict = utils.find_null_columns(source_df=alt_sample_data_df)

alt_sample_data_df = alt_sample_data_df.copy()
data_len_0 = len(alt_sample_data_df)

all_cols = list(alt_sample_data_df.columns)
keep_cols = [x for x in all_cols if x not in global_excluded_cols]
removed_cols = [x for x in all_cols if x not in keep_cols]
alt_sample_data_df = alt_sample_data_df[keep_cols]

alt_sample_data_df, removed_cols = \
    utils.remove_null_cols_by_threshold(source_df=alt_sample_data_df, perc_null_dict=perc_null_dict, null_threshold=null_threshold)

print(f"  >> removed_cols (threshold: {null_threshold}): {removed_cols}")

alt_sample_data_df = alt_sample_data_df.dropna(how="any")#.reset_index(drop=True)
data_len_1 = len(alt_sample_data_df)
print(f"  >> {data_len_0 - data_len_1} records dropped.")

In [ ]:
alt_sample_data_df.shape

In [ ]:
alt_sample_data_df[["Field25"]].value_counts()

In [ ]:
alt_sample_data_df.iloc[:, -21:-19]

In [ ]:
# Generate potential anomaly charts
anomaly_exp = py_caret_anom.setup(data=alt_sample_data_df,
                                  session_id=random_state,
                                  max_encoding_ohe=max_encoding_ohe,
                                  ignore_features=global_excluded_cols,
                                  preprocess=True,
                                  imputation_type="simple",
                                  numeric_imputation="mean",
                                  categorical_imputation="mode",
                                  bin_numeric_features=None,
                                 )

In [ ]:
sub_process_start_time = datetime.now()
print(f" Sub process started @ {sub_process_start_time}")

anomaly_model_list = ["cluster", "iforest"] # , "histogram", "lof", "pca", "mcd"] #, "knn"] # "cof", "svm", "abod", "sod", "sos"
# anomaly_model_list = ["forest"]
tooltip = ["Field17", "Field86", "Field56", "Field52", "Field90"]
# tooltip = ["TruckKey", "DiggerTimeCat1", "CycleOperationCost", "HaulDistance", "EmptyDistance", "CycleTime"]

tooltip_alt = [x for x in tooltip if x in alt_sample_data_df.columns]
tooltip_alt = [x for x in tooltip_alt if x not in global_excluded_cols]
print("tooltip_alt :", tooltip_alt)
    
tooltip_alt = [x for x in tooltip_alt if x not in removed_cols]
print("tooltip_alt :", tooltip_alt)

anomalies_return_dict, anomalies_combined_chart = \
    utils.test_many_anomaly_models(source_df=alt_sample_data_df, 
                                   pycaret_exp=anomaly_exp,
                                   anomaly_model_list=anomaly_model_list,
                                   tooltip=tooltip_alt,
                                   color_scheme="plasma", # plasma turbo yellowgreenblue goldorange
                                   color_column="Anomaly", 
                                   data_name="Records of X",
                                   data_name_suffix="Characteristics",
                                   opacity=0.25,
                                   anomaly_chart_cols=2,
                                   anomaly_chart_width=400,
                                   anomaly_chart_height=400,
                                   anomaly_fractions=anomaly_fractions)

anomalies_combined_chart.display()

sub_process_end_time = datetime.now()
sub_process_duration = sub_process_end_time - sub_process_start_time 
print(f"Sub process ended @ {sub_process_end_time}")
print(f"sub_process_duration {sub_process_duration}")

In [ ]:
tooltip = ["Field17", "Field86", "Field56", "Field52", "Field90"]
# tooltip = ["TruckKey", "DiggerTimeCat1", "CycleOperationCost", "HaulDistance", "EmptyDistance", "CycleTime"]

tooltip_alt = [x for x in tooltip if x in alt_sample_data_df.columns]
tooltip_alt = [x for x in tooltip_alt if x not in global_excluded_cols]   
tooltip_alt = [x for x in tooltip_alt if x not in removed_cols]

try:
    cluster_anomaly_predictions_df, cluster_anomaly_chart, cluster_anomaly_model = \
        utils.test_anomaly_model(source_df=alt_sample_data_df, 
                                 pycaret_exp=anomaly_exp,
                                 model_type="cluster", 
                                 fraction=anomaly_fraction, 
                                 size_col=None,
                                 color_column="Anomaly", 
                                 data_name="Records of X",
                                 data_name_suffix="Characteristics",
                                 color_scheme=color_scheme, 
                                 opacity=0.15,
                                 tooltip=tooltip_alt,
                                 anomaly_chart_width=750,
                                 anomaly_chart_height=750,
                                 random_state=random_state)

    cluster_anomaly_chart.display()
except:
    print(f"  >> Could not create anomaly model.")

In [ ]:
anomaly_X_transformed = py_caret_anom.get_config("X_transformed")

In [ ]:
sub_process_start_time = datetime.now()
print(f" Sub process started @ {sub_process_start_time}")

try:
    print(f"  >> Generating Lime explanations...")
    anomaly_lime_dict, anomaly_lime_single_record_exp_list, \
        lime_anomaly_predictions, lime_anomaly_predictions_alt, lime_anomaly_model, lime_anomaly_experiment, lime_anomaly_explainer = \
        utils.get_lime_explanations_for_anomalies(transformed_data_df=anomaly_X_transformed, 
                                                  model_type="cluster",
                                                  lime_num_features=lime_num_features,
                                                  anomaly_fraction=anomaly_fraction,
                                                  anomaly_kernel_width=None)
except:
    print(f"  >> Could not generate Lime explanations.")
    anomaly_lime_dict = {}
    anomaly_lime_single_record_exp_list = []

sub_process_end_time = datetime.now()
sub_process_duration = sub_process_end_time - sub_process_start_time 
print(f"Sub process ended @ {sub_process_end_time}")
print(f"sub_process_duration {sub_process_duration}")

In [ ]:
positive_influence_on_anomaly_list = []
positive_influence_on_anomaly_dict = {}

for id_key, anom_dict in anomaly_lime_dict.items():
    temp_positive_influence_on_anomaly = anom_dict["positive_influence_on_anomaly"]
    positive_influence_on_anomaly_list.extend(temp_positive_influence_on_anomaly)
    positive_influence_on_anomaly_dict[id_key] = temp_positive_influence_on_anomaly
    

positive_influence_on_anomaly_counter = Counter(positive_influence_on_anomaly_list)
positive_influence_on_anomaly_counter.most_common(10)

anomaly_rank_rename_dict = {x: "Lime_Rank_" + str(x) for x in range(0, lime_num_features)}
anomaly_rank_rename_dict

positive_influence_on_anomaly_df = pd.DataFrame.from_dict(positive_influence_on_anomaly_dict, orient="index")
positive_influence_on_anomaly_df = positive_influence_on_anomaly_df.rename(columns=anomaly_rank_rename_dict)

positive_influence_on_anomaly_df


anomaly_rank_rename_list = [y for _, y in anomaly_rank_rename_dict.items()]
anomaly_rank_rename_list

cluster_anomaly_predictions_with_lime_df = cluster_anomaly_predictions_df.copy()
cluster_anomaly_predictions_with_lime_df = \
    cluster_anomaly_predictions_with_lime_df.merge(positive_influence_on_anomaly_df, how="left", left_index=True, right_index=True)
cluster_anomaly_predictions_with_lime_df = \
    cluster_anomaly_predictions_with_lime_df.merge(lime_anomaly_predictions\
                                                   .rename(columns={"Anomaly": "Lime_Anomaly", "Anomaly_Score": "Lime_Anomaly_Score"})[["Lime_Anomaly", "Lime_Anomaly_Score"]]\
                                                   , how="left", left_index=True, right_index=True)

cluster_anomaly_predictions_with_lime_df[anomaly_rank_rename_list] = cluster_anomaly_predictions_with_lime_df[anomaly_rank_rename_list].fillna("-")
cluster_anomaly_predictions_with_lime_df


lime_tool_tip_cols = ["Anomaly_Score", "Anomaly", 
                      "Lime_Anomaly_Score", "Lime_Anomaly", 
                      "Field17", "Field86", "Field56", "Field52", "Field90"]
# lime_tool_tip_cols = ["Anomaly_Score", "Anomaly", 
#                       "Lime_Anomaly_Score", "Lime_Anomaly", 
#                       "CycleOperationCost",
#                       "CycleTime", "DiggerTimeCat1", "TruckTimeCat1", "TruckGeneralProduction", "Empty", "QueueAtDigger"]

lime_tool_tip_cols_alt = [x for x in lime_tool_tip_cols if x not in global_excluded_cols]
print("lime_tool_tip_cols_alt :", lime_tool_tip_cols_alt)
lime_tool_tip_cols_alt = [x for x in lime_tool_tip_cols_alt if x not in removed_cols]
lime_tool_tip_cols_alt.extend(anomaly_rank_rename_list)

utils.plot_anomaly_chart(anomaly_df=cluster_anomaly_predictions_with_lime_df,
                         dim_0_col="UMAP_0",
                         dim_1_col="UMAP_1",
                         data_name="Records for X",
                         data_name_suffix="Anomalies",
                         size_col=None,
                         color_column="Anomaly",
                         color_scheme=color_scheme,
                         model_type="lime",
                         anomaly_chart_width=750,
                         anomaly_chart_height=750,
                         tool_tip_cols=lime_tool_tip_cols_alt)

In [ ]:
cluster_anomaly_predictions_df

In [ ]:
top_item_num = 5
other_str = "Other"

In [ ]:
cluster_anomaly_predictions_df_alt, cluster_anomaly_int_cols, cluster_anomaly_str_cols, cluster_anomaly_time_cols, cluster_anomaly_float_cols = \
    utils.prepare_data_columns(source_anomaly_df=cluster_anomaly_predictions_df, top_item_num=top_item_num, other_str=other_str)

cluster_anomaly_predictions_df_alt

In [ ]:
cluster_anomaly_predictions_df_alt#["Anomaly"]#.value_counts()

In [ ]:
target = "Anomaly_Mapped"
# ignore_features.extend(["Anomaly_Score", "UMAP_0", "UMAP_1"])
ignore_features = ["Anomaly", "Anomaly_Score", "UMAP_0", "UMAP_1"]
cluster_anomaly_predictions_df_alt[target] = cluster_anomaly_predictions_df_alt["Anomaly"].map({"0": "Normal", "1": "Anomaly"})
ignore_features_alt = ignore_features.copy()
ignore_features_alt.extend(global_excluded_cols)

# fix_imbalance = True
# preprocess = True
# data_split_stratify = True
# fold_shuffle = True

numeric_features = cluster_anomaly_float_cols.copy()
numeric_features = [x for x in numeric_features if x not in ignore_features]
numeric_features = [x for x in numeric_features if x not in [target]]

categorical_features = cluster_anomaly_int_cols.copy()
categorical_features.extend(cluster_anomaly_str_cols)
categorical_features = [x for x in categorical_features if x not in ignore_features]
categorical_features = [x for x in categorical_features if x not in [target]]

date_features = cluster_anomaly_time_cols.copy()
date_features = [x for x in date_features if x not in ignore_features]
date_features = [x for x in date_features if x not in [target]]


# cluster_anomaly_predictions_df_alt_test = cluster_anomaly_predictions_df_alt.copy()
# test_cols = categorical_features.copy()
# test_cols = test_cols[:1]
# test_cols.append(target)

# cluster_anomaly_predictions_df_alt_test = cluster_anomaly_predictions_df_alt_test[test_cols]
# cluster_anomaly_predictions_df_alt_test[target] = cluster_anomaly_predictions_df_alt_test[target].map({"0": "Normal", "1": "Anomaly"})

In [ ]:
chart_cols = 3
width = 250
height = 250

temp_row_charts = []
temp_col_charts = []

for numeric_feature in tqdm(numeric_features):

    temp_chart = alt.Chart(cluster_anomaly_predictions_df_alt)\
                    .mark_bar(opacity=0.5, color="magenta")\
                    .encode(x=alt.X(numeric_feature, 
                                    bin=alt.Bin(maxbins=50),
                                    title=numeric_feature),
                            y="count()")\
                    .properties(title=numeric_feature,
                                width=width,
                                height=height)
    
    temp_row_charts.append(temp_chart)
    
    if len(temp_row_charts) == chart_cols:
        temp_row = alt.hconcat(*temp_row_charts)
        temp_col_charts.append(temp_row)
        temp_row_charts = []

if len(temp_row_charts) > 0:
    temp_row = alt.hconcat(*temp_row_charts)
    temp_col_charts.append(temp_row)
    temp_row_charts = []

alt.vconcat(*temp_col_charts)

In [ ]:
cluster_anomaly_predictions_df_alt.columns

In [ ]:
class_exp = py_caret_class.setup(data=cluster_anomaly_predictions_df_alt,
                                 session_id=random_state,
                                 target=target,
                                 numeric_features=numeric_features,
                                 categorical_features=categorical_features,
                                 date_features=date_features,

                                 # preprocess=preprocess,
                                 # data_split_stratify=data_split_stratify,
                                 # fix_imbalance=fix_imbalance,
                                 # fold_shuffle=fold_shuffle,
                                 bin_numeric_features=None,
                                 ignore_features=ignore_features_alt)

In [ ]:
class_exp_conf_df = pd.DataFrame(class_exp._display_container[0])
class_target_mapping = class_exp_conf_df[class_exp_conf_df['Description'] == 'Target mapping']['Value'].values

target_mapping_dict = dict()
for item in class_target_mapping[0].split(", "):
    label, encoding = item.split(": ")
    target_mapping_dict[encoding] = label

target_mapping_dict

In [ ]:
class_best_model = py_caret_class.compare_models(sort="auc")

In [ ]:
py_caret_class.plot_model(class_best_model, plot='auc')

In [ ]:
dt_class_model = py_caret_class.create_model("dt", max_depth=2)

In [ ]:
class_names = [target_mapping_dict[str(x)] for x in dt_class_model.classes_]

In [ ]:
tree.plot_tree(dt_class_model, 
               feature_names=dt_class_model.feature_names_in_, 
               class_names=class_names,
               filled=True, 
               rounded=True)

In [ ]:
utils.get_rules(tree=dt_class_model, feature_names=dt_class_model.feature_names_in_, class_names=class_names)

In [ ]:
dt_class_dot_data = tree.export_graphviz(dt_class_model, 
                                         feature_names=dt_class_model.feature_names_in_, 
                                         class_names=class_names,
                                         filled=True, 
                                         rounded=True,  
                                         special_characters=True,
                                         out_file=None)
dt_class_graph = graphviz.Source(dt_class_dot_data)
dt_class_graph

In [ ]:
try:
    py_caret_class.plot_model(dt_class_model, plot='auc')
except:
    print(f"  >> Could not plot 'AUC'.")

In [ ]:
ridge_class_model = py_caret_class.create_model("ridge")

In [ ]:
ridge_model_coef_dict = {x: y for x, y in zip(ridge_class_model.feature_names_in_, ridge_class_model.coef_[0])}
ridge_model_coef_dict

ridge_model_coef_df = \
    pd.DataFrame.from_dict(ridge_model_coef_dict, orient="index")\
                .reset_index()\
                .rename({"index": "feature", 0: "coefficient"}, axis=1)\
                .sort_values(["coefficient"], ascending=False)\
                .reset_index(drop=True)

ridge_model_coef_df

In [ ]:
ridge_class_model.intercept_[0]

In [ ]:
lr_class_model = py_caret_class.create_model("lr")

In [ ]:
lr_model_coef_dict = {x: y for x, y in zip(lr_class_model.feature_names_in_, lr_class_model.coef_[0])}
lr_model_coef_dict

lr_model_coef_df = \
    pd.DataFrame.from_dict(lr_model_coef_dict, orient="index")\
                .reset_index()\
                .rename({"index": "feature", 0: "coefficient"}, axis=1)\
                .sort_values(["coefficient"], ascending=False)\
                .reset_index(drop=True)

lr_model_coef_df

In [ ]:
lr_class_model.intercept_[0]

In [ ]:
nb_class_model = py_caret_class.create_model("nb")

In [ ]:
nb_class_model.feature_names_in_

In [ ]:
nb_class_model.class_count_

In [ ]:
nb_class_model.class_prior_

In [ ]:
end_time = datetime.now()
process_duration = end_time - start_time 
print(f"Process ended @ {end_time}")
print(f"process_duration {process_duration}")